In [60]:
!pip install nltk

You should consider upgrading via the 'c:\users\clasa215b_nt17\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [61]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\CLASA215B_NT17\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [62]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\CLASA215B_NT17\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [63]:
import nltk

In [64]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [65]:
df = pd.read_csv('spam.csv', encoding='latin1')

In [66]:
df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [71]:
from sklearn.base import BaseEstimator, TransformerMixin

In [73]:
class TextNormalizer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
        
    def transform(self, X, y=None):
        X_copy = X.copy()
        for i in range(len(X_copy)):
            X_copy[i] = X_copy[i].lower()
            X_copy[i] = X_copy[i].replace('\n',' ')
            X_copy[i] = X_copy[i].replace('\r',' ')
            X_copy[i] = X_copy[i].strip()
        return X_copy

In [74]:
X = df['v2'].values
y = df['v1'].values

In [75]:
X

array(['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
       'Ok lar... Joking wif u oni...',
       "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
       ..., 'Pity, * was in mood for that. So...any other suggestions?',
       "The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free",
       'Rofl. Its true to its name'], dtype=object)

In [76]:
import nltk
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [88]:
class WordExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, language, tokenize):
        self.language = language
        self.stopwords = stopwords.words(self.language)
        self.tokenize = tokenize
        
    def fit(self, X, y=None):
        general_freq = FreqDist()
        for txt in X:
            freq_dist = FreqDist(self.tokenize(txt))
            general_freq.update(freq_dist)
        self.hapaxes = general_freq.hapaxes()
        return self
    def transform(self, X, y=None):
        X_copy = X.copy()
        for i in range(len(X_copy)):
            X_copy[i] = ' '.join([token for token in self.tokenize(X_copy[i])
                            if token not in self.stopwords and
                            token not in self.hapaxes])
        return X_copy

In [100]:
class ApplyStemmer(BaseEstimator, TransformerMixin):
    def __init__(self, stemmer, tokenize):
        self.stemmer = stemmer
        self.tokenize = tokenize
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X_copy = X.copy()
        for i in range(len(X_copy)):
            X_copy[i] = ' '.join([self.stemmer.stem(token)
                                  for token in self.tokenize(X_copy[i])])
        return X_copy

In [101]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, plot_confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression 

In [102]:
pipe = Pipeline([
    ('Normalization', TextNormalizer()),
    ('Word Extraction', WordExtractor('english', word_tokenize)),
    ('Apply Stemmer', ApplyStemmer(PorterStemmer(), word_tokenize)),
    ('Vectorization', CountVectorizer()),
    ('Model', LogisticRegression())
])

In [103]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [104]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('Normalization', TextNormalizer()),
                ('Word Extraction',
                 WordExtractor(language='english',
                               tokenize=<function word_tokenize at 0x000002058B57EDC8>)),
                ('Apply Stemmer',
                 ApplyStemmer(stemmer=<PorterStemmer>,
                              tokenize=<function word_tokenize at 0x000002058B57EDC8>)),
                ('Vectorization', CountVectorizer()),
                ('Model', LogisticRegression())])

In [113]:

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)

In [105]:
import pickle

In [107]:
pickle.dump(pipe, open('pipe.pkl','wb'))